In [3]:
import yfinance as yf
import numpy as np
import pandas as pd
from scipy.stats import norm

In [7]:
tickers = ['RELIANCE.NS', 'HDFCBANK.NS', 'BAJFINANCE.NS']
start_date = '2022-04-01'
end_date = '2024-04-01'

In [8]:
data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

[*********************100%***********************]  3 of 3 completed


In [9]:
data.head()

,BAJFINANCE.NS,HDFCBANK.NS,RELIANCE.NS
Date,,,
2022-04-01,7349.992676,1451.984863,2434.775879
2022-04-04,7431.895020,1597.376221,2441.972412
2022-04-05,7330.496582,1550.567505,2407.456299
2022-04-06,7289.322754,1495.226318,2401.039062
2022-04-07,7233.017578,1462.349365,2358.684814


__Variance Covariance Method__

In [14]:
returns_pf = data.pct_change().dropna()
returns_pf

,BAJFINANCE.NS,HDFCBANK.NS,RELIANCE.NS
Date,,,
2022-04-04,0.011143,0.100133,0.002956
2022-04-05,-0.013644,-0.029304,-0.014135
2022-04-06,-0.005617,-0.035691,-0.002666
2022-04-07,-0.007724,-0.021988,-0.017640
2022-04-08,0.010480,-0.001385,0.016635
...,...,...,...
2024-03-21,0.009963,0.010272,0.005004
2024-03-22,0.006723,-0.002006,0.002791
2024-03-26,0.022068,-0.012094,-0.009244


In [25]:
investment = 175000
weights = np.array([0.3, 0.4, 0.3])


pf_mean = np.dot(weights, returns.mean())
pf_cov = np.dot(weights.T, np.dot(returns.cov(), weights))
# pf_mean
# pf_cov

pf_std = np.sqrt(pf_cov)
#pf_std

In [28]:
confidence_intervals = [0.99, 0.95, 0.90]

VaR = {}
for confidence_interval in confidence_intervals:
    z_value = norm.ppf(confidence_interval)
    VaR[confidence_interval] = z_value * pf_std * investment
    
#print(VaR)

{0.99: 4386.271385532105, 0.95: 3101.33083611178, 0.9: 2416.3337838498474}


In [29]:
for confidence_level, var in VaR.items():
    print("VaR at {}% confidence interval: ₹{:.2f}".format(confidence_level * 100, var))

VaR at 99.0% confidence interval: ₹4386.27
VaR at 95.0% confidence interval: ₹3101.33
VaR at 90.0% confidence interval: ₹2416.33


__Historical Method__

In [34]:
pf_returns = returns.dot(weights)
pf_returns = np.sort(pf_returns)
#pf_returns

In [47]:
def historical_var(returns, confidence_intervals):
    index = int((1 - confidence_intervals) * len(pf_returns))
    return pf_returns[index]

historical_VaR = {}

for confidence_interval in confidence_intervals:
    var = historical_var(pf_returns, confidence_interval)
    historical_VaR[confidence_interval] = -var * investment  
    
#print(historical_VaR)

In [49]:
for confidence_interval in confidence_intervals:
    print(f"Historical_VaR at {int(confidence_interval*100)}% confidence interval: {VaR[confidence_interval]:.2f} rupees")

Historical_VaR at 99% confidence interval: 4386.27 rupees
Historical_VaR at 95% confidence interval: 3101.33 rupees
Historical_VaR at 90% confidence interval: 2416.33 rupees


__Monte Carlo Simulation Method__

In [53]:
num_simulations = 10000
num_days = 1
mean_returns = returns.mean()
cov_matrix = returns.cov()

In [54]:
simulated_pf_returns = []

for _ in range(num_simulations):
    random_returns = np.random.multivariate_normal(mean_returns, cov_matrix, num_days)
    simulated_return = np.dot(random_returns, weights)
    simulated_pf_returns.append(simulated_return)

simulated_pf_returns = np.array(simulated_pf_returns)

In [59]:
def monte_carlo_var(simulated_returns, confidence_intervals):
    index = int((1 - confidence_interval) * len(pf_returns))
    return pf_returns[index]

monte_carlo_VaR = {}

for confidence_interval in confidence_intervals:
    var = monte_carlo_var(simulated_pf_returns, confidence_interval)
    monte_carlo_VaR[confidence_interval] = -var * investment
    
#print(monte_carlo_VaR)

In [60]:
for confidence_interval in confidence_intervals:
    print(f"monte_carlo_VaR at {int(confidence_interval*100)}% confidence interval: {VaR[confidence_interval]:.2f} rupees")

monte_carlo_VaR at 99% confidence interval: 4993.06 rupees
monte_carlo_VaR at 95% confidence interval: 3103.66 rupees
monte_carlo_VaR at 90% confidence interval: 2263.07 rupees
